# 트럼프 트위터 크롤링

In [4]:
import GetOldTweets3 as got
from bs4 import BeautifulSoup
import datetime

days_range = [] # 날짜 범위를 담을 list 생성

start = datetime.datetime.strptime("2017-01-01", "%Y-%m-%d") # datetime 라이브러리 크롤링 시작 날짜 설정
end = datetime.datetime.strptime("2017-01-31", "%Y-%m-%d") # datetime 라이브러리 크롤링 끝나는 날짜 설정
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]
# 시작 날짜와 0부터 끝나는 날짜와의 차이를 추가함
# timedelta 함수는 추가함

for date in date_generated: # 시작 날짜부터 끝나는 날짜까지 for문 돌린다
    days_range.append(date.strftime("%Y-%m-%d")) # days_range 리스트에 시작날짜부터 끝나는 날짜까지 추가

print("=== 설정된 트윗 수집 기간은 {} 에서 {} 까지 입니다 ===".format(days_range[0], days_range[-1])) # 시작날짜~끝나는 날짜 출력
print("=== 총 {}일 간의 데이터 수집 ===".format(len(days_range))) # 데이터 수집 기간 출력

# 특정 검색어가 포함된 트윗 검색하기 (quary search)
# 검색어 : 어벤져스, 스포

import time

# 수집 기간 맞추기
start_date = days_range[0] # days_range의 0번 째 인덱스는 시작 날짜
end_date = (datetime.datetime.strptime(days_range[-1], "%Y-%m-%d")  # day_range의 마지막 인덱스에
            + datetime.timedelta(days=1)).strftime("%Y-%m-%d")  # 1일을 더해줌
# setUntil이 끝을 포함하지 않으므로, day + 1

# 트윗 수집 기준 정의
tweetCriteria = got.manager.TweetCriteria().setUsername('@realDonaldTrump')
# GetOldTweet3 클래스 의 객체 got의 setUsername 함수 사용해서 계정 명 
                                           .setSince(start_date)
# GetOldTweet3 클래스 의 객체 got의 setUsername 함수 사용해서 시작날짜
                                           .setUntil(end_date)
# GetOldTweet3 클래스 의 객체 got의 setUsername 함수 사용해서 끝나는 날짜
                                           

# 수집 with GetOldTweet3
print("Collecting data start.. from {} to {}".format(days_range[0], days_range[-1])) # 수집 시작하면 시작일부터 종료일까지 출력
start_time = time.time() # 시작 시간을 컴퓨터의 현재 시각으로 설정

tweet = got.manager.TweetManager.getTweets(tweetCriteria) # getoldtweet3 클래스의 getTweets 함수로 수집 시작

print("Collecting data end.. {0:0.2f} Minutes".format((time.time() - start_time)/60)) # 시작 시간과 현재 시간의 차이를 수행 시간으로 출력
print("=== Total num of tweets is {} ===".format(len(tweet))) # 수집한 tweet의 갯수 출력


# 원하는 변수 골라서 저장하기

from random import uniform
from tqdm import tqdm_notebook

# initialize
tweet_list = [] # tweet_list 생성

for index in tqdm_notebook(tweet): # tweet의 진행 상황 표시해주는 tqdm_notebook 
    
    # 메타데이터 목록 
    username = index.username # tweet의 username 저장
    content = index.text # tweet의 text 저장
    tweet_date = index.date.strftime("%Y-%m-%d") # tweet의 날짜를 문자열로 저장
    tweet_time = index.date.strftime("%H:%M:%S") # tweet의 시간을 문자열로 저장
    
    
    # 결과 합치기
    info_list = [tweet_date, tweet_time, username, content] # info_list라는 변수에 날짜,시간,유저이름,트윗 내용을 리스트화하여 저장
    tweet_list.append(info_list) # tweet_list 리스트에 info_list 추가함
    
    # 휴식 
  #  time.sleep(uniform(1,2))

# 파일 저장하기

import pandas as pd

twitter_df = pd.DataFrame(tweet_list,  # tweet_list 로 twitter_df 라는 dateframe 생성
                          columns = ["date", "time", "user_name", "text"])

# csv 파일 만들기
twitter_df.to_csv("sample_twitter_data_{}_to_{}.csv".format(days_range[0], days_range[-1]), index=False)
# twitter_df를 sample_twitter_data_시작일_to_종료일.csv로 저장
print("=== {} tweets are successfully saved ===".format(len(tweet_list)))
# tweet 갯수 tweets are succesfully saved 출력


# 파일 확인하기

df_tweet = pd.read_csv('sample_twitter_data_{}_to_{}.csv'.format(days_range[0], days_range[-1]))
# pandas의 read_csv 함수로 twitter_df 파일 읽어옴
df_tweet.head(10) # 위에서 10개만 출력

# 트럼프 크롤링 데이터 전처리

In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

# 데이터 불러오기

df = pd.read_csv('./Trumph_17_19.csv')

# 언어 처리

text_new = []
for i in range(len(df)):
    df1 = df.iloc[i:i+1,]
    text_list = df1['text'].tolist()

    content_text=''
    for each_line in text_list:
        content_text = content_text + str(each_line) + '\n'
    content_text = content_text.replace('\n','')
    content_text = content_text.replace("\\",'') 
    content_text = content_text.replace("\'",'"')   
    
    remove_list=["haven't", 'haven"t', "hadn't", 'hadn"t', "hasn't", 'hasn"t',
                 "didn't", 'didn"t', "doesn't", 'doesn"t', "don't", 'don"t',
                 "wasn't", 'wasn"t', "weren't", 'weren"t', "isn't", 'isn"t',
                 "aren't", 'aren"t', "wouldn't", 'wouldn"t', "won't", 'won"t',
                 "couldn't", 'couldn"t', "shouldn't", 'shouldn"t', "needn't",
                 'needn"t', "mightn't", 'mightn"t', "mustn't", 'mustn"t', "ain't",
                 'ain"t', "shan't", 'shan"t', "can't", 'can"t']
    
    for i in remove_list:
        if i in content_text:
            content_text = content_text.replace(i, 'not')
    
    only_english = re.sub('[^a-zA-Z]', ' ', content_text)
    
    while '  ' in only_english:
        only_english = only_english.replace('  ', ' ')
        
    no_capitals = only_english.lower().split()
    
    stops = set(stopwords.words('english'))
    stops.add('www')
    stops.add('https')
    stops.remove('not')
    
    no_stops = [word for word in no_capitals if not word in stops]
    
    stemmer = nltk.stem.SnowballStemmer('english')
    stemmer_words = [stemmer.stem(word) for word in no_stops]
    
    text_join = ' '.join(stemmer_words)
    text_new.append(text_join)

df['text_new'] = text_new

# TIMES 크롤링

In [ ]:
#Times 데이터 수집
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import time
import pandas as pd

URL='https://www.nytimes.com/search?dropmab=true&endDate=20200102&query=trumph%20&sections=World%7Cnyt%3A%2F%2Fsection%2F70e865b6-cc70-5181-84c9-8368b3a5c34b&sort=oldest&startDate=20170609&types=article'
driver=webdriver.Chrome('chromedriver.exe')
driver.get(URL)

title_list=[] ; date_list =[]; subtitle_list=[]


while ('Dec 31, 2017' not in date_list):
    
    for i in range(4):
        #기사 제목 수집
        Title = driver.find_elements_by_class_name('css-2fgx4k')
    
        time.sleep(0.05)
        #기사 날짜 수집    
        date = driver.find_elements_by_class_name('css-17ubb9w')
    
        time.sleep(0.05)
        #기사 머릿말 수집
        subtitle = driver.find_elements_by_class_name('css-16nhkrn')
    
        time.sleep(0.05)
    
        for i in range(len(Title)):
            if(Title[i].text not in title_list):
                    
                title_list.append(Title[i].text)
                date_list.append(date[i].text)
                subtitle_list.append(subtitle[i].text)
    
        Title.clear()
        date.clear()
        subtitle.clear()
    
        #스크롤 내리기
        elem= driver.find_element_by_tag_name('body')
        elem.send_keys(Keys.PAGE_DOWN)
    
    time.sleep(0.5)
    #SHOW MORE 클릭
    driver.find_element_by_xpath('//*[@id="site-content"]/div/div[2]/div[2]/div/button').click()
    
d=date_list.copy()
t=title_list.copy()
st=subtitle_list.copy()

#기사 제목, 머릿말 결합
for k in range(len(t)):
    t[k]=t[k]+' '+st[k]
    
dc={'date':d, 'text':t}

data = pd.DataFrame(dc)
data.to_csv('20180101_20180630.csv', mode='w')

#Times 데이터 불러오기
one = pd.read_csv('20170101_20170630.csv')
two = pd.read_csv('20170608_20171231.csv')
three = pd.read_csv('20180101_20180701.csv')
four = pd.read_csv('20180701_20181231.csv')
five = pd.read_csv('20190101_20190630.csv')
six = pd.read_csv('20190605_20191231.csv', header=None)

import pandas as pd
import numpy as np
import datetime
from dateutil.parser import parse

#기사 날짜 정규 날짜 표현으로 바꿈
def change_date_form(date):
    if(date[:3]=='Jan'):
        return (date[-4:]+'-'+'01'+'-'+date[5:7]).replace(',','')
    if(date[:3]=='Feb'):
        return (date[-4:]+'-'+'02'+'-'+date[5:7]).replace(',','')
    if(date[:3]=='Mar'):
        return (date[-4:]+'-'+'03'+'-'+date[6:8]).replace(',','')
    if(date[:3]=='Apr'):
        return (date[-4:]+'-'+'04'+'-'+date[6:8]).replace(',','')
    if(date[:3]=='May'):
        return (date[-4:]+'-'+'05'+'-'+date[4:6]).replace(',','')
    if(date[:3]=='Jun'):
        return (date[-4:]+'-'+'06'+'-'+date[5:7]).replace(',','')
    if(date[:3]=='Jul'):
        return (date[-4:]+'-'+'07'+'-'+date[5:7]).replace(',','')
    if(date[:3]=='Aug'):
        return (date[-4:]+'-'+'08'+'-'+date[5:7]).replace(',','')
    if(date[:3]=='Sep'):
        return (date[-4:]+'-'+'09'+'-'+date[6:8]).replace(',','')
    if(date[:3]=='Oct'):
        return (date[-4:]+'-'+'10'+'-'+date[5:7]).replace(',','')
    if(date[:3]=='Nov'):
        return (date[-4:]+'-'+'11'+'-'+date[5:7]).replace(',','')
    if(date[:3]=='Dec'):
        return (date[-4:]+'-'+'12'+'-'+date[5:7]).replace(',','')  

#Times 데이터 하나로 결합
data=pd.concat([one,two,three,four,five,six])

data['date'].apply(change_date_form)
for i in range(len(uu)):
    data['date'][i]=datetime.datetime.strptime(data['date'][i],'%Y-%m-%d').date().strftime('%Y-%m-%d')

# 저장
data.to_csv('Times_data.csv',mode='w')

# TIMES 전처리
- TFIDF 모델 전처리 코드

In [ ]:
# TFIDF 모델 전처리 코드
import pandas as pd
times = pd.read_csv('Times_data_수정.csv')
trump = pd.read_csv('Trumph_Kospi_token.csv')

times=times[['date','text_new']]
times.columns=['date','text']

#형태소 분석 라이브러리
import nltk
nltk.download('wordnet')
from nltk.tokenize import TweetTokenizer ; tknzr = TweetTokenizer()
from nltk.tag import pos_tag
from nltk.stem import SnowballStemmer

def lower_change(text):
    return text.lower()
times['text']=times['text'].apply(lower_change)

Tdate=set(times['date'])
Tdate=sorted(list(Tdate))
text=[]
for j in range(len(Tdate)):
    text.append(str(times[times['date']==Tdate[j]]['text'].values))
for k in range(len(text)):
    text[k] = text[k].replace('[','')
    text[k] = text[k].replace(']','')

dc={'date':Tdate, 'text':text}
Times = pd.DataFrame(dc)

#각 트위터 날짜 기준으로 3일 전, 3일 후의 Times 데이터를 합쳐줌
times_text=[]
timesdate=[s for s in Times['date']]
for x in range(len(trump['date'])):
    
    before = (parse(trump['date'][x])-datetime.timedelta(days=2)).strftime('%Y-%m-%d')
    after = (parse(trump['date'][x])+datetime.timedelta(days=2)).strftime('%Y-%m-%d')
    box_text=[]
    Ttime=trump['date'][x]
    
    #3일 전 데이터가 있으면 3일 전 부터 해당 트위터 날짜까지 Times데이터를 가져옴
    if( before in timesdate or (parse(before)-datetime.timedelta(days=1)).strftime('%Y-%m-%d') in timesdate):
        while(Ttime not in timesdate):
            Ttime=(parse(Ttime)-datetime.timedelta(days=1)).strftime('%Y-%m-%d') 
        
        while(before not in timesdate):
            before=(parse(before)-datetime.timedelta(days=1)).strftime('%Y-%m-%d')
        
        for y in range(timesdate.index(before), timesdate.index(Ttime)+1):
            box_text.append(Times['text'][y])
           
    #3일 전 데이터가 없으면 해당 트위터 날짜 전까지의 Times데이터를 가져옴
    else:
        while(Ttime not in timesdate):
            Ttime=(parse(Ttime)-datetime.timedelta(days=1)).strftime('%Y-%m-%d') 
            
        for z in range(timesdate.index(Ttime)+1):
            box_text.append(Times['text'][z])
            
    #3일 후 데이터가 있으면 해당 트위터 날짜부터 3일 후의 Times데이터를 가져옴
    if( after in timesdate or (parse(after)+datetime.timedelta(days=1)).strftime('%Y-%m-%d') in timesdate):
        while(Ttime not in timesdate):
            Ttime=(parse(Ttime)+datetime.timedelta(days=1)).strftime('%Y-%m-%d') 
            
        while(after not in timesdate):
            after=(parse(after)-datetime.timedelta(days=1)).strftime('%Y-%m-%d')
        
        for yy in range(timesdate.index(after), timesdate.index(Ttime)+1):
            box_text.append(Times['text'][yy])
            
    #2일 후 데이터가 없으면 해당 트위터 날짜부터 끝까지 Times데이터를 가져옴
    else:
        while(Ttime not in timesdate):
            Ttime=(parse(Ttime)+datetime.timedelta(days=1)).strftime('%Y-%m-%d') 
            
        for zz in range(timesdate.index(Ttime),len(timesdate)):
            box_text.append(Times['text'][zz])
    
    times_text.append(box_text)

trump_date=[v for v in trump['date']]
Times_2days={'date':trump_date, 'text':times_text}
times_2days=pd.DataFrame(Times_2days)

#불용어 처리
stop_words=['[',']','\'','\"','""','\' ',',','.','\'''','\\',':','-','\' ','n','s',
           '’','t','v','big','mr','mrs','trump','r','p','new','year','day','month',
           'g','week','state','b','j','u','donald','o','k','le','i','hi','other','e','da',
           'st','e','m','full','f','g','h','i','l','q','r','t','x','z','last','order','rt',
           'w','ka','wan','gen','way','y','d','rig','c','ra','ad','much','va','oh',
           'one','level','leader','term','open','ax','eve','d','all','first','second',
           'time','president','part','move','much','mark','act','el','don','side','reach','happen',
           'thought','felt','would','easili','fabl','various','knew','say','came','togeth','explor',
           'make','begin','record','must','set','ask','pundit','general','januari','alreadi','promis',
           'onto','instead','away','becom','com','xbxd','number','mani','far','realli','longer','peopl',
           'rememb','call','across','said','massiv','increas','dem','like','also','let','somebodi','respond',
           'could','perhap','doesn','night','quick','ly','bit','http','busi','optim','surg','soar','gad','com','donaldtrump',
           'lawenforcementappreciationday','facebook','intellig','agenc','tri','use','paid','total','witch','oppon',
           'congrat','step','gone','get','even','made','rate','onto','morn','coupl','pres','wp','believ','twitter',
           'realdonaldtrump','us','consid','lago','mar','three','ago','ag','bayer','pm','wasn','xl','oval','maga','deliv',
           'facebook','html','long','short','two','everyon','like','wow','total','gotten','amaz','secur','assoc','seen','decis',
           'talk','told','oval','fought','fa','today','hj','etc','wwboo','opufn','dr','met','happi','tri','presid','actual',
           'done','dnc','th','got','anoth','met','org','vp','wh','co','mtgs','also','let','br','general','op','ed','te',
           'great','vp','ron','keep','countri','look','news','help','thank','meet','leav','warm','welcom','man','asap','final',
           'enjoy','fl','friday','visit','live','took','come','back','prior','brief','depart','spoke','zone','minut','unit',
           'go','often','always','better','potenti','wait','mile','certain','nan','next','recent','larg','nc','sure']

#stemmer = nltk.stem.SnowballStemmer('english')

for tt in range(len(times_2days['text'])):
    times_2days['text'][tt] = tknzr.tokenize(str(times_2days['text'][tt]).replace('u.s.','usa'))
    times_2days['text'][tt] = [o for o in times_2days['text'][tt] if o not in stop_words]

#명사만 추출
for pos in range(len(times_2days['text'])):
    pos_text=pos_tag(times_2days['text'][pos])
    times_2days['text'][pos] = [q[0] for q in pos_text if q[1]=='NN'or q[1]=='NNS' or q[1]=='JJ']

#트럼프 트윗과 겹치는 Times 데이터를 트윗 text에 추가하여 가중치 부여
for trum in range(len(trump['text'])):
    trump['text'][trum] = tknzr.tokenize(str(trump['text'][trum]).replace('u.s.','usa'))
    trump['text'][trum] = [oo for oo in trump['text'][trum] if oo not in stop_words]
    for trum2 in range(len(times_2days['text'][trum])):
        if(times_2days['text'][trum][trum2] in trump['text'][trum]):
            trump['text'][trum].append(times_2days['text'][trum][trum2])

for plus in range(len(trump['text'])):
    trump['text'][plus] = ' '.join(trump['text'][plus])

trump.to_csv('weighted_Trump_Kospi_ver2.csv',mode='w')


#  필터링 단어장 생성
- times + forbes2000 + NASDAQ + DJIA + S&P500 + NYSE + Rusell2000 + 국가명 + 주요 도시 = deep러닝용 단어장 생성

In [ ]:
import pandas as pd
import nltk
nltk.download('wordnet')
from nltk.tokenize import TweetTokenizer ; tknzr = TweetTokenizer()
from nltk.tag import pos_tag
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

def lower_change(text):
    return str(text).lower()

#나스닥 상장회사 + 포브스 선정 2000대기업 + 산업군명
nasdaq=pd.read_csv('NASDAQ.txt',sep='\t') #출처 datahub
forbes=pd.read_csv('Forbes.csv') #출처 Kaggle Forbes Top 2000 Companies

nasdaq=nasdaq['Description']
nas=[i.split() for i in nasdaq]

stop = ['Inc','Gp','Cp','Com','CS','Grp','Ltd','Corp','I','Co','Corp.','Right','Unit','Hold','A','Inc.','Wt']

company=[]
for i in range(len(nas)):
    box=[]
    for j in nas[i]:
        if(j not in stop):
            box.append(j)
        company.append(' '.join(box).lower())

forbes=forbes[['Company','Sector','Industry']]
forbes_sector=list(forbes['Sector'].apply(lower_change))
forbes_industry=list(forbes['Industry'].apply(lower_change))
forbes_company=[]
fbs=[f.split() for f in forbes['Company']]
for ii in range(len(fbs)):
    box=[]
    for jj in fbs[ii]:
        if(jj not in stop):
            box.append(jj)
        forbes_company.append(' '.join(box).lower())

forbes=forbes_company+forbes_sector+forbes_industry

#미국 주, 도시 
uscities = pd.read_csv('uscities.csv') #출처: simplemaps interactive data
usa=uscities[['city','state_name','county_name']]
city=list(set(usa['city']))
state=list(set(usa['state_name']))
county=list(set(usa['county_name']))
america = city+state+county

for x in range(len(america)):
    america[x]= america[x].lower()

#국가명, 도시명
world = pd.read_csv('worldcities.csv')  #출처: simplemaps interactive data
country=list(set(world['country']))
country[20]='south korea'
country[161]='north korea'

for y in range(len(country)):
    country[y]= country[y].lower()
    
city=list(set(world['city_ascii']))

for z in range(len(city)):
    city[z]= city[z].lower()

#타임즈 키워드
times=pd.read_csv('Times_data.csv')
times['text']=times['text'].apply(lower_change)

stop_words=['[',']','\'','\"','""','\' ',',','.','\'''','\\',':','-','\' ','n','s',
           '’','t','v','big','mr','mrs','trump','r','p','new','year','day','month',
           'g','week','state','b','j','u','donald','o','k','le','i','hi','other','e','da',
           'st','e','m','full','f','g','h','i','l','q','r','t','x','z','last','order','rt',
           'w','ka','wan','gen','way','y','d','rig','c','ra','ad','much','va','oh','”','“','‘','“','put in'
           'one','level','leader','term','open','ax','eve','d','all','first','second',
           'time','president','part','move','much','mark','act','el','don','side','reach','happen','need','ms','mr','dr','years','moves'
           'thought','felt','would','easili','fabl','various','knew','say','came','togeth','explor',
           'make','begin','record','must','set','ask','pundit','general','januari','alreadi','promis',
           'onto','instead','away','becom','com','xbxd','number','mani','far','realli','longer','peopl',
           'rememb','call','across','said','massiv','increas','dem','like','also','let','somebodi','respond',
           'could','perhap','doesn','night','quick','ly','bit','http','busi','optim','surg','soar','gad','com','donaldtrump',
           'lawenforcementappreciationday','facebook','intellig','agenc','tri','use','paid','total','witch','oppon',
           'congrat','step','gone','get','even','made','rate','onto','morn','coupl','pres','wp','believ','twitter',
           'realdonaldtrump','us','consid','lago','mar','three','ago','ag','bayer','pm','wasn','xl','oval','maga','deliv',
           'facebook','html','long','short','two','everyon','like','wow','total','gotten','amaz','secur','assoc','seen','decis',
           'talk','told','oval','fought','fa','today','hj','etc','wwboo','opufn','dr','met','happi','tri','presid','actual',
           'done','dnc','th','got','anoth','met','org','vp','wh','co','mtgs','also','let','br','general','op','ed','te',
           'great','vp','ron','keep','countri','look','news','help','thank','meet','leav','warm','welcom','man','asap','final',
           'enjoy','fl','friday','visit','live','took','come','back','prior','brief','depart','spoke','zone','minut','unit','answer',
           'go','often','always','better','potenti','wait','mile','certain','nan','next','recent','larg','nc','sure','spirit','answers',
           'animal','lists','start','men','days','sends','waters','tells','acting','kind','thanks','great','a','b','c','weeks','tweets',
           'likes','view','didn','hopes','words','others','groups','seeks','feels','march','considers','brings','months','hundreds',
           'thousands','caught','puts','names','group','friends','friend','others','moves','the','status','steve','book','see','agree',
           'story','worker','workers','jobs','spirit','will','christmas','happy','great','people','place','good','show','nancy','new','nan']


#stop_words에 해당하지 않는 데이터, 명사만 추출
for tt in range(len(times['text'])):
    times['text'][tt] = tknzr.tokenize(str(times['text'][tt]).replace('u.s.','usa'))
    times['text'][tt] = [o for o in times['text'][tt] if o not in stop_words]
    pos_text=pos_tag(times['text'][tt])
    times['text'][tt] = [q[0] for q in pos_text if q[1]=='NN'or q[1]=='NNS']

company=[com for com in company if com not in stop_words]
america=[ame for ame in america if ame not in stop_words]
country=[cou for cou in country if cou not in stop_words]
city=[cit for cit in city if cit not in stop_words]

times_text=[]
for b in range(len(times['text'])):
    times_text.extend(times['text'][b])
times_text.append('dow')

#중요도 필터링 단어장 생성
twit_check_list= country+forbes+country+city
twit_check_list=list(set(twit_check_list))

twit_check_list.remove('new')

'new' in twit_check_list

# 시각화 코드 - EDA

In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
import nltk
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud
df = pd.read_csv('./Trumph_17_19.csv')

df=df.iloc[:,]
X_list=df['text'].tolist()

content_text=''
for each_line in X_list:
    content_text = content_text + str(each_line) + '\n'
    
content_text = content_text.replace('\n','')
content_text = content_text.replace("\\",'') 
content_text = content_text.replace("\'",'"')

only_english = re.sub('[^a-zA-Z]', ' ', content_text)
while '  ' in only_english:
    only_english = only_english.replace('  ', ' ')

no_capitals = only_english.lower().split()      

stops = set(stopwords.words('english'))
stops.add('www')
stops.add('https')

no_stops = [word for word in no_capitals if not word in stops]

no_stops_txt = nltk.Text(no_stops) 

plt.figure(figsize=(16,6))
no_stops_txt.plot(10)
plt.show()

data = no_stops_txt.vocab().most_common(300)
wordcloud = WordCloud(font_path='c:/Windows/Fonts/malgun.ttf',
                     relative_scaling=0.2, background_color='white'
                     ).generate_from_frequencies(dict(data))
plt.figure(figsize=(16,8))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

# 모델링 ( 순환 컨볼루션 레이어)

In [4]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout, Activation
from keras.layers import Conv1D,MaxPooling1D
from keras.preprocessing import sequence  # 기사별 단어 숫자 통일
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.pipeline import Pipeline
import keras
from keras.utils import np_utils
from keras.models import load_model
# 기타 라이브러리
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

# 토크나이저
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.tokenize import TweetTokenizer
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
import re
from wordcloud import WordCloud, STOPWORDS
from nltk.corpus import stopwords
nltk.download('stopwords')

#모델 저장 라이브러리
from keras.models import load_model
import pickle

#언어 분석 라이브러리
from sklearn.pipeline import Pipeline #모델을 한꺼번에 저장해두고 사용
from sklearn.metrics import accuracy_score # accuracy_score(y_test,y_pred) : y_test와 y_pred를 비교하고 값이 같은 비율, 즉 정확도를 계산
from sklearn.model_selection import train_test_split
import sklearn

#워닝 없애기
import warnings
warnings.filterwarnings('ignore')

print('Keras Version : ', keras.__version__)
print('Tensorflow Version : ', tf.__version__)
print('Numpy Version : ', np.__version__)
print('Matplotlib Version : ', matplotlib.__version__)
print('Pandas Version : ', pd.__version__)
print('sklearn Version : ', sklearn.__version__)
print('Nltk Version : ', nltk.__version__)

df = pd.read_csv('C:/Users/kosmo-20/Desktop/Project/code/data_final_1.csv')
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords') # stopwords 클래스를 사용 하기 위해 다운로드

def text_transform(df): 
    text_new = [] # text_new 리스트 생성
    for i in range(len(df)): # 데이터의 길이만큼 for문 돌림
        df1 = df.iloc[i:i+1,] # df1 이라는 변수에 df의 각 행을 저장
        text_list = df1['text'].tolist() # text_list 변수에 df1의 ''text칼럼'값 리스트 저장

        content_text='' # content_text 라는 str타입 변수 생성
        for each_line in text_list: # text_list 에서 for문
            content_text = content_text + str(each_line) + '\n' # content_text에 str타입으로 한 줄 씩 구분하여 추가
        content_text = content_text.replace('\n','')  # \n제거
        content_text = content_text.replace("\\",'')  # \\ 제거
        content_text = content_text.replace("\'",'"')  # \' 제거
    
        remove_list=["haven't", 'haven"t', "hadn't", 'hadn"t', "hasn't", 'hasn"t',
                     "didn't", 'didn"t', "doesn't", 'doesn"t', "don't", 'don"t',
                     "wasn't", 'wasn"t', "weren't", 'weren"t', "isn't", 'isn"t',
                     "aren't", 'aren"t', "wouldn't", 'wouldn"t', "won't", 'won"t',
                     "couldn't", 'couldn"t', "shouldn't", 'shouldn"t', "needn't",
                     'needn"t', "mightn't", 'mightn"t', "mustn't", 'mustn"t', 
                     "ain't", 'ain"t', "shan't", 'shan"t', "can't", 'can"t']
                     # remove_list라는 변수에 부정어 저장
    
        for i in remove_list: # 부정어 list에서 for문
            if i in content_text: #  부정어 list 안에 있는 value가 content_text에 있으면
                content_text = content_text.replace(i , 'not') # content_text에 있는 부정어를 not으로 통일
    
        only_english = re.sub('[^a-zA-Z]', ' ', content_text) # only_english라는 변수에 content_text의 영어 알파벳 형식만 저장
    
        while '  ' in only_english: # only_english에 \t 가 있으면
            only_english = only_english.replace('  ', ' ') # 화이트 스페이스로 계속 교체함
        
        no_capitals = only_english.lower().split() # only_english를 소문자화해서 공백으로 분할한 값을 no_caplitals에 저장
    
        stops = set(stopwords.words('english')) # stopwords 클래스의 객체 stops 생성
        stops.add('www') # stops에 'www' 추가
        stops.add('https') # stops에 'https' 추가
        stops.remove('not') # stops에 not 제거 -> not은 제거하지 말고 살림
    
        no_stops = [word for word in no_capitals if not word in stops]
        # no_capitals의 value들 중 stops의 value와 겹치지 않는 values를 no_stops에 저장함
    
        stemmer = nltk.stem.SnowballStemmer('english') # stem 어간 추출
        stemmer_words = [stemmer.stem(word) for word in no_stops] # 불용어 처리가 끝난 text를 저장한 no_stops에 있는 values의 어간을 추출하여 list에 넣어줌
    
        text_join = ' '.join(stemmer_words) # stemmer_words 를 합쳐줌 
        text_new.append(text_join) # text_new에 text_join을 추가함

    df['text'] = text_new # 불용어 처리가 끝난 text_new를 기존 df의 text 칼럼에 저장
    
    
text_transform(df)

# 모델 구축
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Embedding, Dropout, Conv1D, MaxPooling1D, LSTM, Dense, Flatten,Bidirectional,BatchNormalization,Concatenate

def Model_deep(data):
    index_fund = ['KRX_건설','KRX_경기소비재','KRX_기계장비','KRX_미디어엔터',
                  'KRX_반도체','KRX_방송통신','KRX_보험','KRX_에너지','KRX_운송',
                  'KRX_유틸리티','KRX_은행','KRX_자동차','KRX_정보기술','KRX_증권',
                  'KRX_철강','KRX_필수소비재','KRX_헬스','KRX300_금융',
                  'KRX300_산업재','KRX300_소재','KRX300_자유소비재',
                  'KRX300_정보기술','KRX300_커뮤','KRX300_필수소비재','KRX300_헬스']
    # index_fund에 krx섹터지수 리스트 저장
    Model_Acc = [] # Model_Acc라는 리스트 생성
    tok = Tokenizer(num_words=1000)  # 빈도 수 상위 1000개 단어를 대상으로 하는 Tokenizer 클래스 객체 생성
  
    for A in index_fund:
        df=data[['text',A]] # df에 text칼럼과 krx섹터지수 칼럼 저장
        tok.fit_on_texts(df['text']) # text 칼럼 values들에 인덱스를 추가함
        sequences = tok.texts_to_sequences(df['text']) # 위의 작업 결과를 토대로 벡터화하여 sequences라는 변수에 저장
        
        #train set, test set 구분
        target_list = df[A].tolist() # df의 krx섹터지수 칼럼들의 values를 target_list라는 변수에 저장
        text_train, text_test, target_train, target_test = train_test_split( # sequences를 text_train,test로, target_list를 target_train,test로 데이터 분리)
            sequences,target_list, test_size=0.3,random_state=0) 
          
        #데이터 개수 맞춰주기, 553개 행의 각 데이터 개수를 220개로 지정
        text_train = sequence.pad_sequences(text_train, maxlen=220) # 데이터를 220개 단어로 잘라서 2차원 numpy 배열로 만들어줌
        text_test = sequence.pad_sequences(text_test, maxlen=220)

        # One_hot_encoding
        # target_train = np_utils.to_categorical(target_train) # target_train 벡터(integer) 값을 2진 매트릭스로 변환 -> multi classfication -> softmax -> categorical_crossentropy
        # target_test = np_utils.to_categorical(target_test) # target_test 벡터(integer) 값을 2진 매트릭스로 변환

        #모델 생성
        model = Sequential() # 층 쌓는 모델 생성
        model.add(Embedding(1000,64,input_length=220)) # 벡터크기를 지정해주지 않아도 자동으로 적절한 크기를 찾아줌. Embedding(max_feature, 벡터크기, maxlen)
        # 단어를 의미론적 기하공간에 매핑할 수 있도록 벡터화시킵니다.
        # 첫번째 인자(input_dim) : 단어 사전의 크기를 말하며 총 20,000개의 단어 종류가 있다는 의미입니다. 이 값은 앞서 imdb.load_data() 함수의 num_words 인자값과 동일해야 합니다.
        # 두번째 인자(output_dim) : 단어를 인코딩 한 후 나오는 벡터 크기 입니다.  이 값이 128이라면 단어를 128차원의 의미론적 기하공간에 나타낸다는 의미입니다. 
        # 단순하게 빈도수만으로 단어를 표시한다면, 10과 11은 빈도수는 비슷하지만 단어로 볼 때는 전혀 다른 의미를 가지고 있습니다. 하지만 의미론적 기하공간에서는 거리가 가까운 두 단어는 의미도 유사합니다. 
        # 즉 임베딩 레이어는 입력되는 단어를 의미론적으로 잘 설계된 공간에 위치시켜 벡터로 수치화 시킨다고 볼 수 있습니다.
        # input_length : 단어의 수 즉 문장의 길이를 나타냅니다. 임베딩 레이어의 출력 크기는 샘플 수 * output_dim * input_lenth가 됩니다. 
        # 임베딩 레이어 다음에 Flatten 레이어가 온다면 반드시 input_lenth를 지정해야 합니다. 플래튼 레이어인 경우 입력 크기가 알아야 이를 1차원으로 만들어서 Dense 레이어에 전달할 수 있기 때문입니다.
        # model.add(Dropout(0.5))
        model.add(Conv1D(64,5, padding='same', activation='relu', strides=1))
        # 필터로 특징을 추출하는 컨볼루션 레이어
        # 1번 인자 : 컨볼루션 필터의 수
        # 2번 인자 : 커널의 크기
        # padding : 경계 처리 방법 (valid or same)
        # input_shape : 샘플 수를 제외한 입력 형태, 첫 레이어일 때만 정의 (행,열,채널수) 흑백: 채널=1 컬러:채널=3
        model.add(MaxPooling1D(pool_size=3))
        # 컨볼루션 레이어의 출력 이미지에서 주요 값만 뽑아 크기가 작은 출력 영상을 만듦. 지역적인 사소한 변화의 영향 방지
        model(add(LSTM(64)))
        # ex) LSTM(3, input_dim=1, input_length=4)
        # 1번 인자 : 메모리 셀 갯수, 기억 용량의 정도와 출력 형태 결정
        # input_dim : 입력 속성 수, 일반적으로 속성의 갯수가 들어감
        # input_length : 시퀀스 데이터의 입력 길이 (여기서는 220)
        model.add(Dense(1, activation='sigmoid'))
        # output layer는 binary(0,1), sigmoid
        #학습
        model.compile(loss='binary_crossentropy', optimizer='adam',  # binary_crossentropy 대신 focal 혹은 다른 것들도 고려해볼 것
                      metrics=['accuracy'])
        model.fit(text_train, target_train, batch_size = 100, 
                  validation_split=(0.1), epochs = 15, verbose=0)
        Acc = model.evaluate(text_test, target_test)[1] # 정확도 평가
    
        url = 'Sector_' + A + '.h5'
    
        print('Model : ', url)
        print('Accuracy : %.4f' %Acc)
       
        model.save(url)
        print('저장 완료')
    
    
Model_deep(df)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kosmo-20\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kosmo-20\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\kosmo-20\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kosmo-20\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kosmo-20\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Keras Version :  2.2.4
Tensorflow Version :  1.15.0
Numpy Version :  1.17.4
Matplotlib Version :  3.1.1
Pandas Version :  0.25.3
sklearn Version :  0.22.1
Nltk Version :  3.4.5


ValueError: Layer bidirectional_2 was called with an input that isn't a symbolic tensor. Received type: <class 'keras.engine.sequential.Sequential'>. Full input: [<keras.engine.sequential.Sequential object at 0x0000019EA991E888>]. All inputs to the layer should be tensors.

# 오늘의 twit 전처리

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def text_transform_today(tweet_today):
    remove_list=["haven't", 'haven"t', "hadn't", 'hadn"t', "hasn't", 'hasn"t',
                 "didn't", 'didn"t', "doesn't", 'doesn"t', "don't", 'don"t',
                 "wasn't", 'wasn"t', "weren't", 'weren"t', "isn't", 'isn"t',
                 "aren't", 'aren"t', "wouldn't", 'wouldn"t', "won't", 'won"t',
                 "couldn't", 'couldn"t', "shouldn't", 'shouldn"t', "needn't",
                 'needn"t', "mightn't", 'mightn"t', "mustn't", 'mustn"t', "ain't",
                 'ain"t', "shan't", 'shan"t', "can't", 'can"t']
    for i in remove_list:
        if i in tweet_today:
            content_text = tweet_today.replace(i, 'not')
        else:
            content_text = tweet_today
    
    only_english = re.sub('[^a-zA-Z]', ' ', content_text)
    
    while '  ' in only_english:
        only_english = only_english.replace('  ', ' ')
        
    no_capitals = only_english.lower().split()
    
    stops = set(stopwords.words('english'))
    stops.add('www')
    stops.add('https')
    stops.add('nan')
    stops.remove('not')
    
    no_stops = [word for word in no_capitals if not word in stops]
    
    stemmer = nltk.stem.SnowballStemmer('english')
    stemmer_words = [stemmer.stem(word) for word in no_stops]
    
    text_join = ' '.join(stemmer_words)
    return text_join

In [ ]:
import pandas as pd
df2020 = pd.read_csv('./data_final_2020 (1).csv')
tweet_today = df2020['text']

transform_tweet = text_transform_today(tweet_today)
transform_tweet

# 모델 Predict

In [ ]:
import nltk
nltk.download('stopwords')
import re
from wordcloud import STOPWORDS
from nltk.corpus import stopwords

def transform():
    only_english = re.sub('[^a-zA-Z]', ' ', tweet_today)
    no_capitals = only_english.lower().split()
    stops = set(stopwords.words('english'))
    remove_list=["haven't", "wouldn't", "couldn't", 'ain', 'couldn', 'shouldn',
                 "hadn't", 'doesn', 'mightn', "doesn't", 'don', 'mustn', 'haven',
                 'isn', 'weren', "mightn't", 'didn', "isn't", 'not', "shouldn't", 
                 "hasn't", 'aren', "wasn't", 'wasn', "shan't", 'hasn', "don't", 
                 "didn't", "mustn't", 'hadn', 'wouldn', 'needn', "weren't", 
                 "needn't", "won't", "aren't"]
    for i in remove_list:
        stops.remove(i)
    stops.add('www')
    stops.add('https')
    no_stops = [word for word in no_capitals if not word in stops]
    stemmer = nltk.stem.SnowballStemmer('english')
    stemmer_words = [stemmer.stem(word) for word in no_stops]
    text_join = ' '.join(stemmer_words)
    return text_join



def Using_model():
  Sector = pd.DataFrame(columns= ['Sector','Accuracy','result'])
  
  index_fund = ['KRX_건설','KRX_경기소비재','KRX_기계장비','KRX_미디어엔터',
                'KRX_반도체','KRX_방송통신','KRX_보험','KRX_에너지','KRX_운송',
                'KRX_유틸리티','KRX_은행','KRX_자동차','KRX_정보기술','KRX_증권',
                'KRX_철강','KRX_필수소비재','KRX_헬스','KRX300_금융','KRX300_산업재',
                'KRX300_소재','KRX300_자유소비재','KRX300_정보기술','KRX300_커뮤',
                'KRX300_필수소비재','KRX300_헬스']

  text = input('또람프 트위터 메세지 : ')  # 인풋 메세지
  string = list(transform(text))

  for i in index_fund:
    url = 'Sector_' + i + '.h5'
    
    model = load_model(url)

    # 예측 정확도(r1)
    r1 = np.max(model.predict_proba(string)*100) # 확률 값에 *100 해줌
        
    # 예측 결과(r2)
    r2 = model.predict(string)[0]  # 긍정 --> 1, 부정 --> 0이 출력
        
    if r2 == 1: #긍정이냐?(4~5점)
      df_1= pd.DataFrame({'Sector': [i] ,'Accuracy':[r1],'result' : ['1']})
      Sector = Sector.append(df_1)
    else :        #부정이냐?(1~3점)
      df_2= pd.DataFrame({'Sector': [i] ,'Accuracy': [r1],'result' : ['0']})
      Sector = Sector.append(df_2)
  return Sector

# 텔레그램 전송

In [ ]:
def sending_text():
    data=Using_model()
    up = data.loc[data['result']=='1',['Sector','Accuracy']]
    up = up.sort_values(by=['Accuracy'], axis=0, ascending=False)
    up = up[:3]
    down = data.loc[data['result']=='0', ['Sector','Accuracy']]
    down = down.sort_values(by=['Accuracy'], axis=0, ascending=True)
    down = down[:3]
    up_list = []
    down_list = []
    for i in range(3):
        up_list.append("{0}".format(up.iloc[i,0]))
    for l in range(3):
        down_list.append("{0}".format(down.iloc[l,0]))
    down_str = '\n'.join(down_list)
    up_str = '\n'.join(up_list)

    import datetime
    aaa=str(datetime.datetime.now().date().strftime('%Y-%m-%d')) # 날짜를 str타입 출력
    text_message = aaa + "\n 새로운 트윗입니다\n"+"트윗내용 : \n" + tweet_today+'\n **상승 종목**\n'+up_str+"\n **하강 종목**\n"+down_str            
    return text_message

import telegram

def Telegram():
    text_message = sending_text()
    telgm_token = ['939273649:AAF0ZaO_itBE1zUlgxAiUebOZReb-J_Dthk']
    # 각자 Bot의 API KEY
    # 각자 Bot을 트럼프카드 채널에 입장시킨 후 관리자 등록 후 동작
    bot = telegram.Bot(token = telgm_token) #  봇key를 변수 bot에 저장
    updates = bot.getUpdates() # 롱-폴링 기법을 사용하여 업데이트를 받음
    bot.sendMessage(chat_id='@trumph_card', text=text_message)
    #chat_id = 채팅 룸의 chat_id, text="보낼 메세지 입력"


# 번외) TFIDF 모델

In [ ]:
def Model(data):
    index_fund = ['KRX_건설','KRX_경기소비재','KRX_기계장비','KRX_미디어엔터',
                'KRX_반도체','KRX_방송통신','KRX_보험','KRX_에너지','KRX_운송',
                'KRX_유틸리티','KRX_은행','KRX_자동차','KRX_정보기술','KRX_증권',
                'KRX_철강','KRX_필수소비재','KRX_헬스','KRX300_금융','KRX300_산업재',
                'KRX300_소재','KRX300_자유소비재','KRX300_정보기술','KRX300_커뮤',
                'KRX300_필수소비재','KRX300_헬스']
    
    for A in index_fund:
        df=data[['text',A]]
        text_list = df['text'].tolist()
        target_list = df[A].tolist()

        text_train, text_test, target_train, target_test = train_test_split(text_list,target_list, test_size=0.2,random_state=0) 
        
        tfidf = TfidfVectorizer(lowercase = False, tokenizer = tok)
    
        logistic = LogisticRegression(C= 10,penalty= 'l2', random_state= 0 )

        pipe = Pipeline([('vect',tfidf),('clf',logistic)])
    
        pipe.fit(text_train,target_train)

        y_pred = pipe.predict(text_test)
        
        url = 'Sector_' + A + '.h5'
        
        print('Model : ', url)
        print(accuracy_score(target_test, y_pred))
        
        with open(url,'wb') as fp:
            pickle.dump(pipe,fp)
            
            

def Using_model():
    Sector = pd.DataFrame(columns= ['Sector','Accuracy','result'])
  
    index_fund = ['KRX_건설','KRX_경기소비재','KRX_기계장비','KRX_미디어엔터',
                'KRX_반도체','KRX_방송통신','KRX_보험','KRX_에너지','KRX_운송',
                'KRX_유틸리티','KRX_은행','KRX_자동차','KRX_정보기술','KRX_증권',
                'KRX_철강','KRX_필수소비재','KRX_헬스','KRX300_금융','KRX300_산업재',
                'KRX300_소재','KRX300_자유소비재','KRX300_정보기술','KRX300_커뮤',
                'KRX300_필수소비재','KRX300_헬스']

    text = input('또람프 트위터 메세지 : ')  # 인풋 메세지
    string = [text]

    for i in index_fund:
        url = 'Sector_' + i + '.h5'
        with open(url,'rb') as fp:
              pipe = pickle.load(fp)

        # 예측 정확도(r1)
        r1 = np.max(pipe.predict_proba(string)*100) # 확률 값에 *100 해줌
        
        # 예측 결과(r2)
        r2 = pipe.predict(string)[0]  # 긍정 --> 1, 부정 --> 0이 출력
        
        if r2 == 1: #긍정이냐?(4~5점)
            df_1= pd.DataFrame({'Sector': [i] ,'Accuracy':[r1],'result' : ['1']})
            Sector = Sector.append(df_1)
        else :        #부정이냐?(1~3점)
            df_2= pd.DataFrame({'Sector': [i] ,'Accuracy': [r1],'result' : ['0']})
            Sector = Sector.append(df_2)
    return Sector.to_csv('Sector_predict.csv', mode='w')

# 번외) 컨볼루션  모델

In [ ]:
max_words=1000
tk = data[['text','KRX_건설']]
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(tk['text'])
sequences = tokenizer.texts_to_sequences(tk['text'])
target_list = data['KRX_건설'].tolist()

# 원-핫-=해싱 기법
#ont_hot_results = tokenizer.texts_to_matrix(tk['text'],mode='binary')
# 고유 토큰 찾기
word_index = tokenizer.word_index


# 가능한 토큰의 갯수 ( 단어 인덱스 최댓값 +1), 입베딩차원(여기서는 64)
# embedding_layer = Embedding(1000,64) 

max_features = 1000 # 특성으로 사용할 단어의 수
maxlen=220 # 사용할 텍스트의 길이(가장 빈번한 max_features 개의 단어만 사용)

# 데이터 스플릿
x_train, x_test, y_train, y_test = train_test_split(sequences, target_list, test_size=0.3, random_state=0)

# 리스트를 (sample, maxlen) 크기의 2D 정수 텐서로 변환
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

from keras.models import Sequential
from keras.layers import Flatten,Dense,Embedding

model = Sequential()
# 나중에 임베딩된 입력을 Flatten에서 펼치기 위해 Embedding 층에 input_length 지정
model.add(Embedding(max_features,16,input_length=maxlen))
model.add(Conv1D(32,5, activiation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['acc'])

history = model.fit(x_train, y_train, epochs=10, batch_size=10, validation_data=(x_test,y_test))
# 모델 평가
print("Accuracy :",model.evaluate(x_test,y_test)[1])
# 모델 저장
model.save('Embedding1.txt')

# 번외) 순환 신경망 모델 

In [5]:
# 모델 구축
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Embedding, Dropout, Conv1D, MaxPooling1D, LSTM, Dense, Flatten,Bidirectional,BatchNormalization,Concatenate

def Model_deep(data):
    index_fund = ['KRX_건설','KRX_경기소비재','KRX_기계장비','KRX_미디어엔터',
                  'KRX_반도체','KRX_방송통신','KRX_보험','KRX_에너지','KRX_운송',
                  'KRX_유틸리티','KRX_은행','KRX_자동차','KRX_정보기술','KRX_증권',
                  'KRX_철강','KRX_필수소비재','KRX_헬스','KRX300_금융',
                  'KRX300_산업재','KRX300_소재','KRX300_자유소비재',
                  'KRX300_정보기술','KRX300_커뮤','KRX300_필수소비재','KRX300_헬스']
    # index_fund에 krx섹터지수 리스트 저장
    Model_Acc = [] # Model_Acc라는 리스트 생성
    tok = Tokenizer(num_words=1000)  # 빈도 수 상위 1000개 단어를 대상으로 하는 Tokenizer 클래스 객체 생성
  
    for A in index_fund:
        df=data[['text',A]] # df에 text칼럼과 krx섹터지수 칼럼 저장
        tok.fit_on_texts(df['text']) # text 칼럼 values들에 인덱스를 추가함
        sequences = tok.texts_to_sequences(df['text']) # 위의 작업 결과를 토대로 벡터화하여 sequences라는 변수에 저장
        
        #train set, test set 구분
        target_list = df[A].tolist() # df의 krx섹터지수 칼럼들의 values를 target_list라는 변수에 저장
        text_train, text_test, target_train, target_test = train_test_split( # sequences를 text_train,test로, target_list를 target_train,test로 데이터 분리)
            sequences,target_list, test_size=0.3,random_state=0) 
          
        #데이터 개수 맞춰주기, 553개 행의 각 데이터 개수를 220개로 지정
        text_train = sequence.pad_sequences(text_train, maxlen=220) # 데이터를 220개 단어로 잘라서 2차원 numpy 배열로 만들어줌
        text_test = sequence.pad_sequences(text_test, maxlen=220)

        # One_hot_encoding
        # target_train = np_utils.to_categorical(target_train) # target_train 벡터(integer) 값을 2진 매트릭스로 변환 -> multi classfication -> softmax -> categorical_crossentropy
        # target_test = np_utils.to_categorical(target_test) # target_test 벡터(integer) 값을 2진 매트릭스로 변환

        #모델 생성
        model = Sequential() # 층 쌓는 모델 생성
        model.add(Embedding(1000,64,input_length=220)) # 벡터크기를 지정해주지 않아도 자동으로 적절한 크기를 찾아줌. Embedding(max_feature, 벡터크기, maxlen)
        # 단어를 의미론적 기하공간에 매핑할 수 있도록 벡터화시킵니다.
        # 첫번째 인자(input_dim) : 단어 사전의 크기를 말하며 총 20,000개의 단어 종류가 있다는 의미입니다. 이 값은 앞서 imdb.load_data() 함수의 num_words 인자값과 동일해야 합니다.
        # 두번째 인자(output_dim) : 단어를 인코딩 한 후 나오는 벡터 크기 입니다.  이 값이 128이라면 단어를 128차원의 의미론적 기하공간에 나타낸다는 의미입니다. 
        # 단순하게 빈도수만으로 단어를 표시한다면, 10과 11은 빈도수는 비슷하지만 단어로 볼 때는 전혀 다른 의미를 가지고 있습니다. 하지만 의미론적 기하공간에서는 거리가 가까운 두 단어는 의미도 유사합니다. 
        # 즉 임베딩 레이어는 입력되는 단어를 의미론적으로 잘 설계된 공간에 위치시켜 벡터로 수치화 시킨다고 볼 수 있습니다.
        # input_length : 단어의 수 즉 문장의 길이를 나타냅니다. 임베딩 레이어의 출력 크기는 샘플 수 * output_dim * input_lenth가 됩니다. 
        # 임베딩 레이어 다음에 Flatten 레이어가 온다면 반드시 input_lenth를 지정해야 합니다. 플래튼 레이어인 경우 입력 크기가 알아야 이를 1차원으로 만들어서 Dense 레이어에 전달할 수 있기 때문입니다.
        
        model.add(LSTM(64))
        # ex) LSTM(3, input_dim=1, input_length=4)
        # 1번 인자 : 메모리 셀 갯수, 기억 용량의 정도와 출력 형태 결정
        # input_dim : 입력 속성 수, 일반적으로 속성의 갯수가 들어감
        # input_length : 시퀀스 데이터의 입력 길이 (여기서는 220)
        model.add(Dense(1, activation='sigmoid'))
        # output layer는 binary, sigmoid
        
        #학습
        model.compile(loss='binary_crossentropy', optimizer='adam',  # binary_crossentropy 대신 focal 혹은 다른 것들도 고려해볼 것
                      metrics=['accuracy'])
        model.fit(text_train, target_train, batch_size = 100, 
                  validation_split=(0.1), epochs = 15, verbose=0)
        Acc = model.evaluate(text_test, target_test)[1] # 정확도 평가
    
        url = 'Sector_' + A + '.h5'
    
        print('Model : ', url)
        print('Accuracy : %.4f' %Acc)
       
        model.save(url)
        print('저장 완료')

# 번외) 바나다우 어텐션 메커니즘을 적용한 양방향 순환 신경망 모델 

In [ ]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)

  def call(self, values, query): # 단, key와 value는 같음
    # hidden shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # we are doing this to perform addition to calculate the score
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, BatchNormalization
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
import os

sequence_input = Input(shape=(220,))
embedded_sequences = Embedding(vocab_size, 128, input_length=max_len)(sequence_input)

lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional \
    (LSTM
     (128,
      dropout=0.3,
      return_sequences=True,
      return_state=True,
      recurrent_activation='relu',
      recurrent_initializer='glorot_uniform'))(embedded_sequences)

state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
state_c = Concatenate()([forward_c, backward_c]) # 셀 상태

attention = BahdanauAttention(128) # 가중치 크기 정의
context_vector, attention_weights = attention(lstm, state_h)

hidden = BatchNormalization()(context_vector)

output = Dense(2, activation='softmax')(hidden)
model = Model(inputs=sequence_input, outputs=output)
Adam = optimizers.Adam(lr=0.0001, clipnorm=1.)
model.compile(optimizer=Adam, loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(text_train, target_train, epochs=10, batch_size=128, validation_data=(text_test, target_test), verbose=1)

'C:\\Users\\kosmo-20\\Desktop\\Project\\code'